In [1]:
!pip install langchain_google_genai

import base64
import mimetypes
from google.colab import userdata, files
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableBranch
from langchain_google_genai import ChatGoogleGenerativeAI

try:
    GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
        temperature=0
    )
except Exception as e:
    print(f"Error initializing LLM: {e}. Please check your API Key.")

def image_to_base64(img_path):
    with open(img_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def get_image_data_url(image_path):
    mime_type, _ = mimetypes.guess_type(image_path)
    if mime_type is None:
        mime_type = "image/png"

    encoded_string = image_to_base64(image_path)

    return f"data:{mime_type};base64,{encoded_string}"

In [2]:
def map_extract_data(image_paths):
    all_results = []
    parser = JsonOutputParser()

    for p in image_paths:
        try:
            data_url = get_image_data_url(p)
            prompt_text = (
                "Analyze this receipt image to extract financial data.\n"
                "1. Identify the 'Total Paid' (Final Charge).\n"
                "2. Identify 'Discount': Look for any numeric value with a NEGATIVE SIGN (e.g., -1.00, -5.20). "
                "Sum the ABSOLUTE VALUES of these negative numbers. "
                "Also include items labeled 'Savings' or 'Adjustment' if they reduce the total.\n"
                "If no discount/negative numbers found, set discount to 0.\n"
                "\n"
                "Return ONLY JSON format: {\"total\": float, \"discount\": float}"
            )
            content = [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": data_url}}
            ]

            res = (llm | parser).invoke([HumanMessage(content=content)])

            if isinstance(res, list): res = res[0]

            all_results.append({
                "total": abs(float(res.get('total', 0.0))),
                "discount": abs(float(res.get('discount', 0.0)))
            })

        except Exception as e:
            all_results.append({"total": 0.0, "discount": 0.0})

    return all_results

In [3]:
def reduce_and_route(data_list, query):
    grand_total = sum(item['total'] for item in data_list)
    grand_discount = sum(item['discount'] for item in data_list)
    grand_original = grand_total + grand_discount

    query = query.lower()
    if "total" in query and ("spend" in query or "pay" in query or "cost" in query or "money" in query):
        return f"Your total spending is {grand_total:.2f}"
    elif ("without" in query and "discount" in query) or "original" in query or "gross" in query:
        return (f"You should pay {grand_original:.2f} without discount")
    else:
        return "I am sorry, I can't answer this question."

In [4]:
def process_homework():
    while True:
        print("\n" + "="*50 + "\n[Instruction] Upload images to start analysis | Click 'Cancel upload' to exit")
        uploaded = files.upload()
        paths = list(uploaded.keys())
        if not paths: break

        extracted_data = map_extract_data(paths)

        print("Commands: Enter 'new' to re-upload, 'exit' to quit")
        while True:
            user_input = input("\nYour Query > ").strip()

            if user_input.lower() in ['exit', 'quit']: return
            if user_input.lower() == 'new': break

            response = reduce_and_route(extracted_data, user_input)
            print(f"\nAI Response: {response}")

if __name__ == "__main__":
    process_homework()


[Instruction] Upload images to start analysis | Click 'Cancel upload' to exit


Saving receipt1.jpg to receipt1.jpg
Saving receipt2.jpg to receipt2.jpg
Saving receipt3.jpg to receipt3.jpg
Saving receipt4.jpg to receipt4.jpg
Saving receipt5.jpg to receipt5.jpg
Saving receipt6.jpg to receipt6.jpg
Saving receipt7.jpg to receipt7.jpg
Commands: Enter 'new' to re-upload, 'exit' to quit

Your Query > How much money did I spend in total for these bills？

AI Response: Your total spending is 1974.30

Your Query > How much would I have had to pay without the discount?

AI Response: You should pay 2348.20 without discount

Your Query > What's the weather today?

AI Response: I am sorry, I can't answer this question.

Your Query > exit
